The following code/functions below are helpers for the search algorithms developed later.

In [28]:
from collections import Counter
with open('sgb-words.txt', 'r') as file:
    words = file.read().splitlines()

words_dict = set(words)
letters = [chr(c) for c in range(ord('a'), ord('z') + 1)]

def display(arr):
    return " -> ".join(arr)

The heuristic function that I choose was a function that calculated how many letters are different than the target word.

This heuristic function was choosen since it is simple while also also being admissible and consistant. The reason why it is admissible is because h(n) <= h*(n) where h(n) is our heuristic function and h*(n) is the true value. This is because in our heuristic function we always assume that we only need to change 1 letter at a time to get the correct value. As a result, we always will have a value less than the true value. This means that our A* algorithm will be curious and search all possibilities and get one closest to the true value. The reason why it is consistant is because h(n) <= cost(n, n') + h(n'). This is because the cost will always be 1 and the max that you could possibly change from h(n) to h(n') is 1. To make it simplier, we can represent it as h(n) - h(n') <= cost(n, n') where the max that h(n) - h(n') can be is 1 as you can only change 1 lettter at a time. 

Since our heuristic function is both admissible and consistant, this means that we don't need to reopen our queue in our A* search algorithm.

In [29]:
# Return number of letters away from target word
def heuristic_function(guessed_word: str, target_word: str) -> int:
    if len(guessed_word) != len(target_word):
        raise Exception("Invalid guess, lengths are different")
    n = 0
    for i in range(len(guessed_word)):
        if guessed_word[i] != target_word[i]:
            n += 1
    return n

In [30]:
from collections import deque
def word_ladder_ucs(start, target, word_list):
    dq = deque()
    dq.append([start])
    seen = set()
    seen.add(start)
    num_iterations = 0
    if start == target:
        return [start]
    while dq:
        num_iterations += 1
        curr_array = dq.popleft()
        curr_word = curr_array[-1]
        if curr_word == target:
            return curr_array
        for i in range(len(start)):
            curr_word_list = list(curr_word)
            for c in letters:
                if c == curr_word[i]:
                    continue
                temp_copy_word = curr_word_list[:]
                temp_copy_word[i] = c
                next_word = "".join(temp_copy_word)
                if next_word in word_list:
                    if next_word in seen:
                        continue
                    next_array = curr_array[:]
                    next_array.append(next_word)
                    if next_word == target:
                        return next_array, num_iterations
                    else:
                        dq.append(next_array)
                        seen.add(next_word)

    return num_iterations

In [31]:
import heapq 
def word_ladder_astar(start, target, word_list):
    heap = []
    seen = set()
    seen.add(start)
    num_iterations = 0
    heapq.heappush(heap, (heuristic_function(start, target), [start]))
    if start == target:
        return [start]
    while heap:
        num_iterations += 1
        curr_val = heapq.heappop(heap)
        curr_array = curr_val[1]
        curr_word = curr_array[-1]
        if curr_word == target:
            return curr_array
        for i in range(len(start)):
            curr_word_list = list(curr_word)
            for c in letters:
                if c == curr_word[i]:
                    continue
                temp_copy_word = curr_word_list[:]
                temp_copy_word[i] = c
                next_word = "".join(temp_copy_word)
                if next_word in word_list and next_word not in seen:
                    next_array = curr_array[:]
                    next_array.append(next_word)
                    if next_word == target:
                        return next_array, num_iterations
                    else:
                        heapq.heappush(heap, (heuristic_function(next_word, target) + len(next_array) - 1, next_array))
                        seen.add(next_word)
    return num_iterations

Testing code comparing A* and UCS

In [32]:
def test(start, target):
    print("*********************************************")
    print(f"Testing search algorithm on converting {start} into {target}")
    print()
    astar_res = word_ladder_astar(start, target, words_dict)
    astar_iterations = 0
    if isinstance(astar_res, int):
        print(f"A* failed to find a path")
        astar_iterations = astar_res
    else:
        print(f"A* found path {display(astar_res[0])}")
        astar_iterations = astar_res[1]
    print()
    ucs_res = word_ladder_ucs(start, target, words_dict)
    ucs_iterations = 0
    print()
    if isinstance(ucs_res, int):
        print(f"UCS failed to find a path")
        ucs_iterations = ucs_res
    else:
        print(f"UCS found path {display(ucs_res[0])}")
        ucs_iterations = ucs_res[1]
    print()
    print(f"UCS had {ucs_iterations} iterations while A* had {astar_iterations}.")
    print(f"Thats a difference of {ucs_iterations-astar_iterations}!")
    print("*********************************************")

def start_test(test_tupes):
    for start, end in test_tupes:
        test(start, end)
        print()
        print()

tests_to_run = [("start", "stops"), ("hello", "world"), ("walks", "while"), ("works", "grade")]

start_test(tests_to_run)



*********************************************
Testing search algorithm on converting start into stops

A* found path start -> stare -> store -> stoae -> stoas -> stops


UCS found path start -> stare -> store -> stoae -> stoas -> stops

UCS had 183 iterations while A* had 18.
Thats a difference of 165!
*********************************************


*********************************************
Testing search algorithm on converting hello into world

A* failed to find a path


UCS failed to find a path

UCS had 4493 iterations while A* had 4493.
Thats a difference of 0!
*********************************************


*********************************************
Testing search algorithm on converting walks into while

A* found path walks -> walls -> wails -> waits -> whits -> white -> while


UCS found path walks -> walls -> wails -> waits -> whits -> white -> while

UCS had 1641 iterations while A* had 40.
Thats a difference of 1601!
*********************************************


***